In [13]:
import numpy as np
import scipy as sp
import pandas as pd
from IPython.display import display, HTML

data = pd.read_csv("malware_Binary.csv")
display(HTML(data.head(100).to_html()))



,hash,millisecond,classification,os,state,usage_counter,prio,static_prio,normal_prio,policy,vm_pgoff,vm_truncate_count,task_size,cached_hole_size,free_area_cache,mm_users,map_count,hiwater_rss,total_vm,shared_vm,exec_vm,reserved_vm,nr_ptes,end_data,last_interval,nvcsw,nivcsw,min_flt,maj_flt,fs_excl_counter,lock,utime,stime,gtime,cgtime,signal_nvcsw
0,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914e223349672eca79ad0,0,malware,CentOS,0,0,3069378560,14274,0,0,0,13173,0,0,24,724,6850,0,150,120,124,210,0,120,3473,341974,0,0,120,0,3204448256,380690,4,0,0,0
1,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914e223349672eca79ad0,1,malware,Windows,0,0,3069378560,14274,0,0,0,13173,0,0,24,724,6850,0,150,120,124,210,0,120,3473,341974,0,0,120,0,3204448256,380690,4,0,0,0
2,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914e223349672eca79ad0,2,malware,Mac,0,0,3069378560,14274,0,0,0,13173,0,0,24,724,6850,0,150,120,124,210,0,120,3473,341974,0,0,120,0,3204448256,380690,4,0,0,0
3,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914e223349672eca79ad0,3,malware,Ubuntu,0,0,3069378560,14274,0,0,0,13173,0,0,24,724,6850,0,150,120,124,210,0,120,3473,341974,0,0,120,0,3204448256,380690,4,0,0,0
4,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914e223349672eca79ad0,4,malware,Mac,0,0,3069378560,14274,0,0,0,13173,0,0,24,724,6850,0,150,120,124,210,0,120,3473,341974,0,0,120,0,3204448256,380690,4,0,0,0
5,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914e223349672eca79ad0,5,malware,Windows,0,0,3069378560,14274,0,0,0,13173,0,0,24,724,6850,0,150,120,124,210,0,120,3473,341974,0,0,120,0,3204448256,380690,4,0,0,0
6,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914e223349672eca79ad0,6,malware,Ubuntu,0,0,3069378560,14274,0,0,0,13173,0,0,24,724,6850,0,150,120,124,210,0,120,3473,341974,0,0,120,0,3204448256,380690,4,0,0,0
7,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914e223349672eca79ad0,7,malware,Mac,0,0,3069378560,14274,0,0,0,13173,0,0,24,724,6850,0,150,120,124,210,0,120,3473,341974,0,0,120,0,3204448256,380690,4,0,0,0
8,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914e223349672eca79ad0,8,malware,CentOS,0,0,3069378560,14274,0,0,0,13173,0,0,24,724,6850,0,150,120,124,210,0,120,3473,341974,0,0,120,0,3204448256,380690,4,0,0,0
9,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914e223349672eca79ad0,9,malware,Mac,0,0,3069378560,14274,0,0,0,13173,0,0,25,724,6852,0,150,120,124,211,0,120,3473,341974,0,130,120,0,3204448256,380690,4,0,0,0


In [14]:
# keep 19 columns and remove others which are not appropriate to be included in this classification task
df=data.drop(['hash','millisecond','policy','state','cached_hole_size','free_area_cache'
             ,'mm_users','end_data','last_interval','min_flt','maj_flt','fs_excl_counter'
             ,'lock','utime','stime','gtime','cgtime'],axis=1)

# print out and display dataframe as tables in HTML
display(HTML(df.head(10).to_html()))

,classification,os,usage_counter,prio,static_prio,normal_prio,vm_pgoff,vm_truncate_count,task_size,map_count,hiwater_rss,total_vm,shared_vm,exec_vm,reserved_vm,nr_ptes,nvcsw,nivcsw,signal_nvcsw
0,malware,CentOS,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0
1,malware,Windows,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0
2,malware,Mac,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0
3,malware,Ubuntu,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0
4,malware,Mac,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0
5,malware,Windows,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0
6,malware,Ubuntu,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0
7,malware,Mac,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0
8,malware,CentOS,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0
9,malware,Mac,0,3069378560,14274,0,0,13173,0,6852,0,150,120,124,211,0,341974,0,0


In [5]:
print('Column Datatypes:\n',df.dtypes)

# convert all nominal variables to binary variables
df_raw=df.copy(deep=True) 
df_knn=df.copy(deep=True) 
# create new binary columns
df_dummies=pd.get_dummies(df_knn[['os']])
# add them to dataframe
df_knn=df_knn.join(df_dummies)
# drop original columns
df_knn=df_knn.drop(['os'],axis=1)
display('Data Example:',HTML(df_knn.head(10).to_html()))


Column Datatypes:
 classification       object
os                   object
usage_counter         int64
prio                  int64
static_prio           int64
normal_prio           int64
vm_pgoff              int64
vm_truncate_count     int64
task_size             int64
map_count             int64
hiwater_rss           int64
total_vm              int64
shared_vm             int64
exec_vm               int64
reserved_vm           int64
nr_ptes               int64
nvcsw                 int64
nivcsw                int64
signal_nvcsw          int64
dtype: object


'Data Example:'

,classification,usage_counter,prio,static_prio,normal_prio,vm_pgoff,vm_truncate_count,task_size,map_count,hiwater_rss,total_vm,shared_vm,exec_vm,reserved_vm,nr_ptes,nvcsw,nivcsw,signal_nvcsw,os_CentOS,os_Debian,os_Mac,os_Ubuntu,os_Windows
0,malware,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0,1,0,0,0,0
1,malware,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0,0,0,0,0,1
2,malware,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0,0,0,1,0,0
3,malware,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0,0,0,0,1,0
4,malware,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0,0,0,1,0,0
5,malware,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0,0,0,0,0,1
6,malware,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0,0,0,0,1,0
7,malware,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0,0,0,1,0,0
8,malware,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0,1,0,0,0,0
9,malware,0,3069378560,14274,0,0,13173,0,6852,0,150,120,124,211,0,341974,0,0,0,0,1,0,0


In [16]:
import pandas as pd

# List of columns to exclude from normalization
columns_to_exclude = ['classification','usage_counter', 'normal_prio', 'vm_pgoff', 'task_size', 'hiwater_rss', 'nr_ptes', 'signal_nvcsw'] 

# Filter the columns to normalize
columns_to_normalize = [col for col in df_knn.columns if col not in columns_to_exclude]

# Min-max normalization for selected columns
for col in columns_to_normalize:
    df_knn[col] = (df_knn[col] - df_knn[col].min()) / (df_knn[col].max() - df_knn[col].min())

# Display the DataFrame
display(HTML(df_knn.head(10).to_html()))

,classification,usage_counter,prio,static_prio,normal_prio,vm_pgoff,vm_truncate_count,task_size,map_count,hiwater_rss,total_vm,shared_vm,exec_vm,reserved_vm,nr_ptes,nvcsw,nivcsw,signal_nvcsw,os_CentOS,os_Debian,os_Mac,os_Ubuntu,os_Windows
0,malware,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,1.0,0.0,0.0,0.0,0.0
1,malware,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,0.0,0.0,1.0
2,malware,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,1.0,0.0,0.0
3,malware,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,0.0,1.0,0.0
4,malware,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,1.0,0.0,0.0
5,malware,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,0.0,0.0,1.0
6,malware,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,0.0,1.0,0.0
7,malware,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,1.0,0.0,0.0
8,malware,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,1.0,0.0,0.0,0.0,0.0
9,malware,0,0.18254,0.016007,0,0,0.199175,0,0.166589,0,0.052031,1.0,0.307692,0.250689,0,0.091519,0.0,0,0.0,0.0,1.0,0.0,0.0


In [17]:
# encode label, since KNN requires label encoding
from sklearn import preprocessing

y = df_knn['classification']
le = preprocessing.LabelEncoder()
le.fit(y)
y_encoded = le.transform(y)

print(y_encoded)

df_knn['classification'] = y_encoded
x_features = df_knn.drop('classification',axis=1)

display(HTML(df_knn.head(10).to_html()))

[1 1 1 ... 1 1 1]


,classification,usage_counter,prio,static_prio,normal_prio,vm_pgoff,vm_truncate_count,task_size,map_count,hiwater_rss,total_vm,shared_vm,exec_vm,reserved_vm,nr_ptes,nvcsw,nivcsw,signal_nvcsw,os_CentOS,os_Debian,os_Mac,os_Ubuntu,os_Windows
0,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,1.0,0.0,0.0,0.0,0.0
1,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,0.0,0.0,1.0
2,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,1.0,0.0,0.0
3,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,0.0,1.0,0.0
4,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,1.0,0.0,0.0
5,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,0.0,0.0,1.0
6,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,0.0,1.0,0.0
7,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,1.0,0.0,0.0
8,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,1.0,0.0,0.0,0.0,0.0
9,1,0,0.18254,0.016007,0,0,0.199175,0,0.166589,0,0.052031,1.0,0.307692,0.250689,0,0.091519,0.0,0,0.0,0.0,1.0,0.0,0.0


In [19]:
# Build KNN models and evaluate the models

from sklearn.model_selection import train_test_split
from sklearn import neighbors
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings("ignore")

# 75% as training, 25% as testing
x_train, x_test, y_train, y_test = train_test_split(x_features, y_encoded, test_size=0.25)

kvalue = [25, 50, 75, 100, 125, 150, 175, 200]

metrics = ['euclidean', 'manhattan']
for m in metrics: 
    print('metric: ', m)
    for k in range(25, 200, 25): 
        clf=neighbors.KNeighborsClassifier(k, metric=m)
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        auc = roc_auc_score(y_test, y_pred)
        print('K =', k, ', Accuracy: ', accuracy_score(y_test, y_pred), ', Precision: ', precision_score(y_test, y_pred, average='macro', zero_division=0),
             ', Recall: ', recall_score(y_test, y_pred, average='macro', zero_division=0), ', AUC: ', auc)





metric:  euclidean
K = 25 , Accuracy:  0.99848 , Precision:  0.9984799176577475 , Recall:  0.9984799176577475 , AUC:  0.9984799176577476
K = 50 , Accuracy:  0.99344 , Precision:  0.9934332303581113 , Recall:  0.9934514212661094 , AUC:  0.9934514212661094
K = 75 , Accuracy:  0.98768 , Precision:  0.9876855094239386 , Recall:  0.9877111295168015 , AUC:  0.9877111295168015
K = 100 , Accuracy:  0.98096 , Precision:  0.9809679666463735 , Recall:  0.9809925319730599 , AUC:  0.98099253197306
K = 125 , Accuracy:  0.97292 , Precision:  0.972915631785331 , Recall:  0.9729223604150947 , AUC:  0.9729223604150947
K = 150 , Accuracy:  0.96672 , Precision:  0.9667685908874901 , Recall:  0.9666905220389026 , AUC:  0.9666905220389026
K = 175 , Accuracy:  0.96096 , Precision:  0.9611285382805428 , Recall:  0.9608984130826773 , AUC:  0.9608984130826772
metric:  manhattan
K = 25 , Accuracy:  0.99908 , Precision:  0.9990790989154721 , Recall:  0.9990808334091135 , AUC:  0.9990808334091134
K = 50 , Accuracy